# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [26]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql("SELECT * FROM messages", engine)

In [ ]:
df.head()

In [12]:
X = ['genre', 'message']
Y = list(set(df.columns)-set(['id', 'message', 'original', 'genre']))

In [27]:
df.loc[df['related'] == 2, 'related'] = 1

In [ ]:
for y in Y:
    print(y, df[y].unique())

### 2. Write a tokenization function to process your text data

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /home/yingjie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/yingjie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yingjie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize andremove stop words
    stop_words = stopwords.words("english")
    tokens = [lemmatizer.lemmatize(word).strip() for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
from sklearn.model_selection import train_test_split

In [14]:
X = df[X]
y = df[Y]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [21]:
y_pred = pipeline.predict(X_test)

In [22]:
y_train

,military,fire,storm,missing_people,money,request,food,search_and_rescue,aid_centers,weather_related,...,aid_related,other_weather,earthquake,shelter,buildings,infrastructure_related,electricity,security,child_alone,death
22622,0,0,1,0,0,0,0,0,1,1,...,1,1,1,0,0,1,0,0,0,0
4828,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
12505,0,0,1,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
5027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10898,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
14314,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10342,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
12101,0,0,0,0,0,1,0,0,0,1,...,1,0,0,1,1,0,0,0,0,0


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [56]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
    #'vect__max_features': (None, 500),
    #'clf__estimator__n_estimators': (10, 100),
    'clf__estimator__class_weight': [None, 'balanced', 'balanced_subsample'],
    'vect__binary': (True, False),
}

model = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
%time
model.fit(X_train,y_train)
model.best_params_
y_pred = model.predict(X_test)
print(classification_report(y_pred, y_test))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [40]:
from sklearn.pipeline import Pipeline, FeatureUnion

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
df = pd.concat([pd.get_dummies(df['genre']), df], axis=1)

In [30]:
df.drop(['genre'], axis=1, inplace=True)

In [31]:
df.head()

,direct,news,social,id,message,original,related,request,offer,aid_related,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,1,0,0,1,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
X = df[['direct', 'news', 'social', 'message']]
y = df[Y]

In [39]:
X.head()

,direct,news,social,message
0,1,0,0,Weather update - a cold front from Cuba that c...
1,1,0,0,Is the Hurricane over or is it not over
2,1,0,0,Looking for someone but no name
3,1,0,0,UN reports Leogane 80-90 destroyed. Only Hospi...
4,1,0,0,"says: west side of Haiti, rest of the country ..."


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [41]:
X_test.head()

,direct,news,social,message
7917,1,0,0,"Please ,give me some help.Thanks before."
25322,0,1,0,A positive development is that cooling has bee...
22191,0,1,0,Witnesses say 200 to 500 people have so far di...
18442,0,1,0,The 15-nation Economic Community of West Afric...
1336,1,0,0,and in addition what time will it be possible ...


In [42]:
X_test[:1].values

array([[1, 0, 0, 'Please ,give me some help.Thanks before. ']],
      dtype=object)

In [44]:
query = 'Please ,give me some help.Thanks before. '
genre = 'Direct'

In [46]:
input_df = pd.DataFrame([[query, 0, 0, 0]], columns = ['message', 'direct', 'social', 'news'])

In [47]:
input_df.head()

,message,direct,social,news
0,"Please ,give me some help.Thanks before.",0,0,0


In [48]:
if genre == 'Direct':
    input_df['direct'] = 1
elif genre == 'social':
    input_df['social'] = 1
else:
    input_df['news'] = 1

In [49]:
input_df.head()

,message,direct,social,news
0,"Please ,give me some help.Thanks before.",1,0,0


In [33]:
from sklearn.base import BaseEstimator, TransformerMixin

In [35]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [46]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('selector', ItemSelector(key='message')),
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('genre_enc', ItemSelector(key=['direct', 'social', 'news'])),
    ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

In [43]:
y_pred = pipeline.predict(X_test)

In [44]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [48]:
for i in range(35):
    print(y_test.columns[i])
    print(classification_report(pd.DataFrame(y_pred).iloc[:, i], pd.DataFrame(y_test).iloc[:, i]))

military
             precision    recall  f1-score   support

          0       1.00      0.97      0.98      8612
          1       0.09      0.60      0.16        40

avg / total       0.99      0.97      0.98      8652

fire
             precision    recall  f1-score   support

          0       1.00      0.99      0.99      8650
          1       0.01      0.50      0.02         2

avg / total       1.00      0.99      0.99      8652

storm
             precision    recall  f1-score   support

          0       0.99      0.94      0.96      8247
          1       0.38      0.76      0.51       405

avg / total       0.96      0.93      0.94      8652

missing_people
             precision    recall  f1-score   support

          0       1.00      0.99      0.99      8652
          1       0.00      0.00      0.00         0

avg / total       1.00      0.99      0.99      8652

money
             precision    recall  f1-score   support

          0       1.00      0.98      0.99   

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [45]:
print(classification_report(y_pred, y_test))

             precision    recall  f1-score   support

          0       0.09      0.60      0.16        40
          1       0.01      0.50      0.02         2
          2       0.38      0.76      0.51       405
          3       0.00      0.00      0.00         0
          4       0.02      0.44      0.04         9
          5       0.47      0.82      0.60       843
          6       0.61      0.83      0.70       709
          7       0.12      0.70      0.20        37
          8       0.00      0.00      0.00         0
          9       0.64      0.82      0.72      1862
         10       0.00      0.00      0.00         1
         11       0.09      0.58      0.16       112
         12       0.04      0.60      0.08        20
         13       0.09      0.85      0.16        13
         14       0.34      0.74      0.46       772
         15       0.00      0.00      0.00         0
         16       0.11      0.71      0.19        70
         17       0.26      0.83      0.40   

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [49]:
from xgboost import XGBClassifier

pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('selector', ItemSelector(key='message')),
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('genre_enc', ItemSelector(key=['direct', 'social', 'news'])),
    ])),
        ('clf', MultiOutputClassifier(XGBClassifier())),
    ])

In [50]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('selector', ItemSelector(key='message')), ('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', inp...scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
           n_jobs=1))])

In [52]:
y_pred = pipeline.predict(X_test)

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

In [53]:
print(classification_report(y_pred, y_test))

             precision    recall  f1-score   support

          0       0.19      0.60      0.29        85
          1       0.21      0.56      0.31        34
          2       0.58      0.77      0.66       607
          3       0.16      0.85      0.27        20
          4       0.22      0.64      0.33        66
          5       0.51      0.84      0.64       894
          6       0.77      0.81      0.79       919
          7       0.15      0.70      0.24        47
          8       0.03      0.44      0.06         9
          9       0.61      0.88      0.72      1661
         10       0.00      0.00      0.00         1
         11       0.19      0.62      0.29       211
         12       0.21      0.67      0.32        93
         13       0.45      0.77      0.57        74
         14       0.35      0.79      0.49       755
         15       0.00      0.00      0.00         0
         16       0.25      0.78      0.38       143
         17       0.55      0.87      0.68   

In [54]:
pipeline.get_params()

{'memory': None,
 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('selector', ItemSelector(key='message')), ('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, ...ublinear_tf=False, use_idf=True))])), ('genre_enc', ItemSelector(key=['direct', 'social', 'news']))],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
          max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
          n_estimators=100, n_jobs=1, nthread=None,
          objective='binary:logistic', random_state=0, reg_alpha=0,
          reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
          subsample

In [60]:
parameters = {
    #'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2))
    #'features__text_pipeline__tfidf__sublinear_tf': (True, False), 
    'clf__estimator__max_depth': (3,5,7), #7
    'clf__estimator__subsample': (0.8, 1),#1
}

model = GridSearchCV(pipeline, param_grid=parameters)

In [58]:
%time
model.fit(X_train,y_train)
model.best_params_
y_pred = model.predict(X_test)
print(classification_report(y_pred, y_test))

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 14.5 µs


/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

...

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

             precision    recall  f1-score   support

          0       0.27      0.60      0.37       120
          1       0.22      0.59      0.32        34
          2       0.65      0.74      0.70       713
          3       0.12      0.76      0.21        17
          4       0.27      0.54      0.36        96
          5       0.56      0.81      0.66      1015
          6       0.77      0.81      0.79       917
          7       0.15      0.73      0.26        48
          8       0.03      0.50      0.07         8
          9       0.67      0.86      0.75      1861
         10       0.02      0.50      0.04         2
         11       0.25      0.63      0.35       274
         12       0.23      0.60      0.33       113
         13       0.47      0.76      0.58        79
         14       0.43      0.78      0.55       928
         15       0.00      0.00      0.00         1
         16       0.29      0.73      0.42       179
         17       0.57      0.84      0.68   

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are 

In [71]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__tfidf__sublinear_tf': (True, False),
    #     'clf__estimator__max_depth': (7, 10),
    #     'clf__estimator__colsample_bytree': (0.6, 1)
}

model = GridSearchCV(pipeline, param_grid=parameters)

In [72]:
%time
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, y_test))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.78 µs


/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

...

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

             precision    recall  f1-score   support

          0       0.17      0.56      0.26        81
          1       0.29      0.63      0.40        41
          2       0.56      0.78      0.65       583
          3       0.16      0.85      0.27        20
          4       0.21      0.65      0.32        62
          5       0.51      0.84      0.64       888
          6       0.78      0.81      0.79       939
          7       0.18      0.75      0.29        55
          8       0.05      0.50      0.09        12
          9       0.61      0.88      0.72      1643
         10       0.00      0.00      0.00         2
         11       0.18      0.61      0.28       210
         12       0.22      0.66      0.33        97
         13       0.41      0.74      0.53        70
         14       0.36      0.78      0.49       773
         15       0.00      0.00      0.00         1
         16       0.25      0.81      0.38       137
         17       0.54      0.87      0.66   

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0

In [66]:
model.best_params_

{'clf__estimator__colsample_bytree': 0.6, 'clf__estimator__max_depth': 10}

### 9. Export your model as a pickle file

In [ ]:
# {'clf__estimator__colsample_bytree': 0.6, 'clf__estimator__max_depth': 10}


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.